In [1]:
# """
# View more, visit my tutorial page: https://morvanzhou.github.io/tutorials/
# My Youtube Channel: https://www.youtube.com/user/MorvanZhou
# More about Reinforcement learning: https://morvanzhou.github.io/tutorials/machine-learning/reinforcement-learning/
# Dependencies:
# torch: 0.4
# gym: 0.8.1
# numpy
# """

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym

In [3]:
# Hyper Parameters
BATCH_SIZE = 32
LR = 0.01                   # learning rate
EPSILON = 0.9               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = 2000
env = gym.make('CartPole-v0')
env = env.unwrapped
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample().shape     # to confirm the shape

In [4]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 50)
        self.fc1.weight.data.normal_(0, 0.1)   # initialization
        self.out = nn.Linear(50, N_ACTIONS)
        self.out.weight.data.normal_(0, 0.1)   # initialization

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value

In [5]:
class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()
        print("Neural net")
        print(self.eval_net)

        self.learn_step_counter = 0                                     # for target updating
        self.memory_counter = 0                                         # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # initialize memory
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()

    def choose_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # input only one sample
        if np.random.uniform() < EPSILON:   # greedy
            actions_value = self.eval_net.forward(x)
            action = torch.max(actions_value, 1)[1].data.numpy()
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
        else:   # random
            action = np.random.randint(0, N_ACTIONS)
            action = action if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        return action
    
    def choose_greedy_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # input only one sample
        actions_value = self.eval_net.forward(x)
        action = torch.max(actions_value, 1)[1].data.numpy()
        action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
        return action

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1

    def learn(self):
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # sample batch transitions
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])

        # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [6]:
dqn = DQN()

print('\nCollecting experience...')
for i_episode in range(800):
    s = env.reset()
    ep_r = 0
    while True:
#         env.render()
        a = dqn.choose_action(s)

        # take action
        s_, r, done, info = env.step(a)

        # modify the reward
        x, x_dot, theta, theta_dot = s_
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        r = r1 + r2

        dqn.store_transition(s, a, r, s_)

        ep_r += r
        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
            if done:
                print('Ep: ', i_episode,
                      '| Ep_r: ', round(ep_r, 2))

        if done:
            break
        s = s_

Neural net
Net(
  (fc1): Linear(in_features=4, out_features=50, bias=True)
  (out): Linear(in_features=50, out_features=2, bias=True)
)

Ep:  204 | Ep_r:  2.67
Ep:  205 | Ep_r:  7.67
Ep:  206 | Ep_r:  0.73
Ep:  207 | Ep_r:  2.11
Ep:  208 | Ep_r:  2.41
Ep:  209 | Ep_r:  2.47
Ep:  210 | Ep_r:  1.24
Ep:  211 | Ep_r:  2.34
Ep:  212 | Ep_r:  3.41
Ep:  213 | Ep_r:  2.15
Ep:  214 | Ep_r:  8.54
Ep:  215 | Ep_r:  10.51
Ep:  216 | Ep_r:  4.1
Ep:  217 | Ep_r:  -1.59
Ep:  218 | Ep_r:  4.38
Ep:  219 | Ep_r:  14.41
Ep:  220 | Ep_r:  2.95
Ep:  221 | Ep_r:  2.56
Ep:  222 | Ep_r:  2.65
Ep:  223 | Ep_r:  19.78
Ep:  224 | Ep_r:  53.77
Ep:  225 | Ep_r:  -5.31
Ep:  226 | Ep_r:  6.43
Ep:  227 | Ep_r:  11.7
Ep:  228 | Ep_r:  22.04
Ep:  229 | Ep_r:  7.52
Ep:  230 | Ep_r:  18.27
Ep:  231 | Ep_r:  27.61
Ep:  232 | Ep_r:  141.08
Ep:  233 | Ep_r:  149.88
Ep:  234 | Ep_r:  48.82
Ep:  235 | Ep_r:  85.6
Ep:  236 | Ep_r:  69.42
Ep:  237 | Ep_r:  166.1
Ep:  238 | Ep_r:  465.44
Ep:  239 | Ep_r:  441.02
Ep:  240 | Ep_r:

In [7]:
env.close()

In [8]:
print(torch.__version__)


0.4.1.post2


In [9]:
s = env.reset()
while True:
    env.render()
    a = dqn.choose_greedy_action(s)
    s_, r, done, info = env.step(a)
    if done:
        break
    s = s_

In [10]:
env.close()

In [11]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in dqn.eval_net.state_dict():
    print(param_tensor, "\t",dqn.eval_net.state_dict()[param_tensor].size())


Model's state_dict:
fc1.weight 	 torch.Size([50, 4])
fc1.bias 	 torch.Size([50])
out.weight 	 torch.Size([2, 50])
out.bias 	 torch.Size([2])


In [12]:
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in dqn.optimizer.state_dict():
    print(var_name, "\t", dqn.optimizer.state_dict()[var_name])

Optimizer's state_dict:
state 	 {140200659953272: {'step': 718135, 'exp_avg': tensor([[-0.0000,  0.0000, -0.0000, -0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0000, -0.0000, -0.0000,  0.0000],
        [-0.0000, -0.0000, -0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.0000, -0.0000],
        [ 0.0000,  0.0000, -0.0000, -0.0000],
        [-0.0000,  0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000,  0.0000],
        [ 0.0003, -0.0001, -0.0000,  0.0002],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0013,  0.0007,  0.0001, -0.0008],
        [ 0.0000,  0.0001, -0.0000, -0.0002],
        [-0.0000, -0.0000, -0.0000,  0.0000],
        [-0.0000, -0.0000,  0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.0000, -0.0000],
        [ 0.0000, -0.0000,  0.0000,  0.0000],
        [-0.0004, -0.0000,  0.0000,  0.0001],
  

In [13]:
PATH = "cartpole_weights"
torch.save(dqn.eval_net.state_dict(), PATH)


In [14]:
# model = dqn.eval_net()
# model.load_state_dict(torch.load(PATH))
# model.eval()